In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib 


In [2]:
df= pd.read_csv(r'Bengaluru_House_Data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Bengaluru_House_Data.csv'

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.groupby('area_type')['area_type'].agg('count')

In [ ]:
df = df.drop(['area_type', 'society', 'balcony', 'availability'], axis='columns')

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['size'].unique()

In [ ]:
df['bhk']= df['size'].apply(lambda x: int(x.split(' ')[0]) )

In [ ]:
df.head()

In [ ]:
df['bhk'].unique()

In [ ]:
df[df.bhk>20]

In [ ]:
df.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df[~df['total_sqft'].apply(is_float)].head(10)

In [ ]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+ float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
convert_sqft_to_num('2100 - 2850')

In [ ]:
convert_sqft_to_num('4125Perch')

In [ ]:
df = df.copy()
df['total_sqft']= df['total_sqft'].apply(convert_sqft_to_num)
df.head()

In [ ]:
df.loc[30]

In [ ]:
df = df.copy()
df['price_per_sqft'] = df['price']*100000/df['total_sqft']
df.head()

In [ ]:
len(df.location.unique())

In [ ]:
df.location = df.location.apply(lambda x: x.strip())

location_strats = df.groupby('location')['location'].agg('count').sort_values(ascending = False)
location_strats

In [ ]:
len(location_strats[location_strats<=10])

In [ ]:
location_strats_less_than_10 = location_strats[location_strats<=10]
location_strats_less_than_10

In [ ]:
len(df.location.unique())

In [ ]:
df.location = df.location.apply(lambda x: 'other' if x in location_strats_less_than_10 else x)
len(df.location.unique())

In [ ]:
df.head(10)

In [ ]:
#outliers detection

In [ ]:
df[df.total_sqft/df.bhk<300].head()

In [ ]:
df.shape

In [ ]:
df = df[~(df.total_sqft/df.bhk<300)]
df.shape

In [ ]:
df.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index =True)
    return df_out

df = remove_pps_outliers(df)
df.shape

In [ ]:
def plot_scatter_chart(df, location):
    bhk2 = df[(df.location == location)& (df.bhk==2)]
    bhk3 = df[(df.location == location)& (df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft, bhk2.price, color ='blue', label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft, bhk3.price, marker='+', color ='green', label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price Per Square Feet")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df, "Uttarahalli")

#we should remove properties wherefor same location, the price of 2bhk appartment is less then 2bhk appartment(with same sqft). 
#what we do is for a given loc, we will built a dict of stats per bhk i.e

{
    '1': {
        'mean': 4000,
        'std': 2000,
        'count': 34
    },
    
    '2':{
        'mean': 4300,
        'std': 2300,
        'count': 22
        
    },
}

#now we can remove those 2bhk appartments whose price per sqft is less than mean price per sqft of 1bhk appartment

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats ={}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis='index')

df =remove_bhk_outliers(df)
df.shape

In [ ]:
plot_scatter_chart(df, "Uttarahalli")

In [ ]:
import matplotlib

matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df.price_per_sqft, rwidth = 0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df.bath.unique()

In [ ]:
df[df.bath>10]

In [ ]:
plt.hist(df.bath, rwidth=0.8)
plt.xlabel("No of Bathrooms")
plt.ylabel("Count")

In [ ]:
df[df.bath>df.bhk+2]

In [ ]:
df = df[df.bath<df.bhk+2]
df.shape

In [ ]:
df= df.drop(['size', 'price_per_sqft'], axis= 'columns')
df.head()

In [ ]:
dummies = pd.get_dummies(df.location, dtype=float)
dummies.head(3)

In [ ]:
df = pd.concat([df, dummies.drop('other', axis='columns')], axis='columns')

In [ ]:
df.head()

In [ ]:

df1 = df.drop('location', axis ='columns')
df1.head(3)

In [ ]:
df1.shape

In [ ]:
X= df1.drop('price', axis='columns')
X.head()

In [ ]:
y= df1.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=10)


In [ ]:
from sklearn.linear_model import LinearRegression

lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)
lr_clf.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv= ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
''' from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params':{
                'copy_X' : [True, False],
                'fit_intercept' : [True, False],
                'n_jobs' : [1,2,3],
                'positive' : [True, False]
            }
        },
        'lasso':{
            'model': Lasso(),
            'params': {
                'alpha':[1,2],
                'selection':['random', 'cyclic']
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion':['mse', 'friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores=[]
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs=GridSearchCV(config['model'], config['params'], cv=cv, return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
        
    return pd.DataFrame(scores, columns=['model','best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,y)'''

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'copy_X' : [True, False],
                'fit_intercept' : [True, False],
                'n_jobs' : [1,2,3],
                'positive' : [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(x.columns == location)[0][0]
    
    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath 
    x[3] = bhk
    if loc_index >=0:
        x[loc_index] =1
    return lr_clf.predict([x])[0]

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 2, 2)

In [ ]:
predict_price('1st Phase JP Nagar', 1000, 2, 3)

In [ ]:
predict_price('Indira Nagar', 1000, 3,3)

In [ ]:
predict_price('Indira Nagar', 1000, 2,2)

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle', 'wb') as f:
    pickle.dump(lr_clf, f)

In [ ]:
import json
columns ={
    'data_columns': [col.lower()for col in X.columns]
}

with open("columns.json", 'w') as f:
    f.write(json.dumps(columns))